<a href="https://colab.research.google.com/github/monimoyd/P2_S9/blob/master/T3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Phase2 Assignment 9

In this assignment, I have privided steps for T3D RL algorithm

## install all the modules

In [1]:
!pip install gym

In [0]:
!pip install pybullet

In [0]:
!pip install agents

  Created wheel for agents: filename=agents-1.4.0-cp36-none-any.whl size=62746 sha256=dd21b8203283ef3c001505d4f321c3cac20bcca4d6edb7440b78b5c1b10b8021
  Stored in directory: /root/.cache/pip/wheels/1c/23/05/e3ed899566d6d9e52aa5c0e7c0f454f4e474b7d01cf5ad106b
Successfully built agents


In [0]:
!pip install ruamel.yaml

## Import libraries

In [0]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import pybullet_envs
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from gym import wrappers
from torch.autograd import Variable
from collections import deque


## Create Repplay Buffer Class

- add method of class used for adding new transitions
- sample method of class is used for sampling list of transitions

In [0]:
class ReplayBuffer(object):
    def __init__(self, max_size=1e6):
        self.storage = []
        self.max_size = max_size
        self.ptr = 0
    def add(self, transition):
        if len(self.storage) == self.max_size:
            self.storage[int(self.ptr)] = transition
            self.ptr = (self.ptr + 1) % self.max_size
        else:
            self.storage.append(transition)
    def sample(self, batch_size):
        ind = np.random.randint(0, len(self.storage), batch_size)
        batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = [], [], [], [], []
        for i in ind:
            state, next_state, action,reward,done = self.storage[i]
            batch_states.append(np.array(state, copy=False))
            batch_next_states.append(np.array(next_state, copy=False))
            batch_rewards.append(np.array(reward, copy=False))
            batch_dones.append(np.array(done, copy=False))
            batch_states.append(np.array(state, copy=False))
        return np.array(batch_states), np.array(batch_next_states),  np.array(batch_actions), np.array(batch_rewards).reshape(-1,1), np.array(batch_dones).reshape(-1,1)


## Define Actor class

In [0]:
class Actor(nn.Module):
    def __init__(self, state_dims, action_dims, max_action):
        # max action is to clip in case we added too much noise
        super(Actor, self).__init__()
        self.layer_1 = nn.Linear(state_dims, 400)
        self.layer_2 = nn.Linear(400, 300)
        self.layer_3 = nn.Linear(300, action_dim)
        self.max_action = max_action
    def forward(self, x):
        x = F.relu(self.layer_1(x))
        x = F.relu(self.layer_2(x))
        x = self.max_action * torch.tanh(self.layer_3(x))
        return x

## Define Critic Class


In [0]:
class Critic(nn.Module):
    def __init__(self, state_dims, action_dim):
        self.layer_1 = nn.Linear(state_dims + action_dim, 400)
        self.layer_2 = nn.Linear(400, 300)
        self.layer_3 = nn.Linear(300, action_dim)
        self.layer_4 = nn.Linear(state_dims + action_dim, 400)
        self.layer_5 = nn.Linear(400, 300)
        self.layer_6 = nn.Linear(300, action_dim)

    def forward(self, x, u):
        xu = torch.cat([x, u], 1)
        x1 = F.relu(self.layer_1(xu))
        x1 = F.relu(self.layer_2(x1))
        x1 = self.layer_3(x1)

        x2 = F.relu(self.layer_4(xu))
        x2 = F.relu(self.layer_5(x2))
        x2 = self.layer_6(x2)

        return x1, x2
    
    def Q1(self, x, u):
        xu = torch.cat([x, u], 1)
        x1 = F.relu(self.layer_1(xu))
        x1 = F.relu(self.layer_2(x1))
        x1 = self.layer_3(x1)
        return x1


## Define T3D class
- T3D class creates an instance of Actor and Actor target
- Create instance of Critic1 and Critic1 target, Critic2 and Critic2 target
- taing method is used for training T3D

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class T3D(object):
    def __init__(self, state_dims, action_dim, max_action):
        self.actor = Actor(state_dims, action_dim, max_action).to(device)
        self.actor_target = Actor(state_dims, action_dim, max_action).to(device)
        self.actor_target.load_state_dict(self.actor.state_dict)
        # Initializing with model weights to keep then same
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters())

        self.critic = Critic(state_dims, action_dim).to(device)
        self.critic_target = Critic(state_dims, action_dim).to(device)
        self.critic_target.load_state_dict(self.critic.state_dict)
        self.critic_optimizer = torch.optim.Adam(seld.critic.parameterss())
        self.max_action = max_action

    def select_action(self, state):
        state = torch.tensor(state.reshape(1, -1)).to(device)
        return self.actor(state).cpu.data.numpy().flatten()

    def train(self, replay_buffer, iterations, batch_size=100, discount=0.99, tau = 0.005, policy_noise=0.2, noise_clip=0.5, policy_freq=2):
        for it in range(iterations):
            # Step 4 We sample from a batch of transitions (s, s', a, r) from memory
            batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = replay_buffer.sample(batch_size)
            state = torch.Tensor(batch_states).to(device)
            next_state = torch.Tensor(batch_next_states).to(device)
            action = torch.Tensor(batch_actions).to(device)
            reward = torch.Tensor(batch_rewards).to(device)
            done = torch.Tensor(batch_dones).to(device)
            # Step5: From the next state s', the Actor target plays the next action a'
            next_action = self.actor_target.forward(next_state)

            # Step6: We add Gaussian noise to this next action a' and we clamp it in a 
            # range of values supported by the environment
            noise = torch.Tensor(batch_actions).data.normal_(0, policy_noise).to(device)
            noise = noise.clamp(-noise_clip, noise_clip)
            next_action = (next_action + noise).clamp(-self.max_action, self.max_action)

            # Step7: The two Critic targets take each the couple (s', a') as input and 
            # return two Q values, Qt1(s', a') and Qt2(s', a') as outputs
            target_Q1, target_Q2 = self.critic_target.forward(next_state, next_action)

            # Step8: We keep the minimum of these two Q values
            target_Q = torch.min(target_Q1, target_Q2)

            # Step 9: We get the final target of the two Critic model, whish is:
            # Qt = r + gamma*min(Qt1, Qt2)
            # target_Q = reward + (1-done)* discount * target_Q
            # done 0 means episode not over, 1- episode over
            # We can't run the above equation efficiently as some components are in Computational 
            # graphs and some are not. We need to make one minor modification
            target_Q = reward + ((1-done)* discount *target_Q).detach()

            # Step 10: the two critic models take each the couple (s,a),
            # as input adn return two Q values
            current_Q1, current_Q2 = self.critic.forward(state, action)

            # Step 11: We compute the loss coming from two Critic models
            critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)

            # Step 12: We backpropagate teh Critic loss and update the parameters of the two Critic
            # models with a Adam optimizer
            self.critic_optimizer.zero_grad() # initializing the gradients to zero
            critic_loss.backward() # computing the gradients
            self.crtic_optimizer.step() # performing the wegiht updates

            # Step 13: Once every two iterations, we updte our Actor model by 
            # performing gradient ascent on the output of the first critic model
            if it % policy_freq == 0:
                # This is DPG part
                actor_loss = -(self.critic.Q1(state, self.actor(state)).mean())
                self.actor_optimizer.grad_zero()
                actor_loss.backward()
                self.actor_optimizer.step()

            # Step 14: Still once every two iterations, we update the weights of teh Actor target
            # by Polyak averaging
            for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
                target_param.data.copy_(tau * param.data + ( 1 - tau) * target_param.data)
            

             #Step 15: Still once every tow iterations, we update the weights of the Critic target
             # by Polyak averaging
            for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
                target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

